# Installation

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2 # OpenCV
import mediapipe as mp
import numpy as np

# For drawing
mp_drawing = mp.solutions.drawing_utils

# For pose estimation
mp_pose = mp.solutions.pose

# Determine Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# Calculate Angles

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# Up and Down Counter

## Left Hand

In [4]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for elbow angle
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
            # Get coordinates for shoulder angle
            hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angle
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
            angle_shoulder_left = calculate_angle(hip_left, shoulder_left, elbow_left)
            
            # Visualize angle
            # cv2.putText(image, str(angle_left), 
            #                tuple(np.multiply(elbow_left, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 0), 2, cv2.LINE_AA
            #                     )
            # cv2.putText(image, str(angle_shoulder_left),
            #                 tuple(np.multiply(shoulder_left, [640, 480]).astype(int)),
            #                 cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 0), 2, cv2.LINE_AA
            #                       )
            
            # Curl counter logic
            if angle_left > 160:
                stage = "down"
            if angle_left < 40 and stage =='down':
                stage="up"
                counter +=1
                print(counter)

            # Count if counter = 5 Notify "Please take rest"ArithmeticError
            if counter > 5:
                cv2.putText(image, "Please take rest", 
                           (300,150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 5, (255,0,0), 8, cv2.LINE_AA
                                )
                # Press 's' to reset counter
                if cv2.waitKey(10) & 0xFF == ord('s'):
                    counter = 0

            # Check if left shoulder angle is more than 45 degree notify "low your hand down"
            if angle_shoulder_left > 45:
                cv2.putText(image, "Do not go up too high with your left hand.", 
                           (150,300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
                cv2.putText(image, "Tuck your shoulders in!", 
                           (150,400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'CNT', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Fitful Model Application - Left Hand Exercise', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1705463172.840972       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-01-17 10:46:13.013 Python[14372:353497] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


1
2
3
4
5
6
7
1
2
3
4
5
6
7
1


## Right Hand

In [5]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for elbow angle
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Get coordinates for shoulder angle
            hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angle_shoulder_right = calculate_angle(hip_right, shoulder_right, elbow_right)
            
            # Visualize angle
            # cv2.putText(image, str(angle_right), 
            #                tuple(np.multiply(elbow_right, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 0), 2, cv2.LINE_AA
            #                     )

            # cv2.putText(image, str(angle_shoulder_right), 
            #                tuple(np.multiply(elbow_right, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 0), 2, cv2.LINE_AA
            #                     )
            
            # Curl counter logic
            if angle_right > 160:
                stage = "down"
            if angle_right < 40 and stage =='down':
                stage="up"
                counter +=1
                print(counter)

            # Count if counter = 5 Notify "Please take rest"ArithmeticError
            if counter > 5:
                cv2.putText(image, "Please take rest", 
                           (300,150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 5, (255,0,0), 8, cv2.LINE_AA
                                )
                # Press 's' to reset counter
                if cv2.waitKey(10) & 0xFF == ord('s'):
                    counter = 0

        # Check if left shoulder angle is more than 45 degree notify "low your hand down"
            if angle_shoulder_right > 45:
                cv2.putText(image, "Do not go up too high with your right hand.", 
                           (150,300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
                cv2.putText(image, "Tuck your shoulders in!", 
                           (150,400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )     
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'CNT', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Fitful Model Application - Right Hand Exercise', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1705463228.522475       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


## Both Hands

In [6]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for elbow angle
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Get coordinates for shoulder angle
            hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Calculate angle
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)

            angle_shoulder_left = calculate_angle(hip_left, shoulder_left, elbow_left)
            angle_shoulder_right = calculate_angle(hip_right, shoulder_right, elbow_right)
            
            # Visualize angle
            # cv2.putText(image, str(angle_left), 
            #                tuple(np.multiply(elbow_left, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

            # cv2.putText(image, str(angle_right), 
            #                tuple(np.multiply(elbow_right, [640, 480]).astype(int)), 
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )
            
            # Curl counter logic
            if angle_left > 160 and angle_right > 160:
                stage = "down"
            if angle_left < 40 and angle_right < 40 and stage =='down':
                stage="up"
                counter +=1
                print(counter)

             # Count if counter = 5 Notify "Please take rest"ArithmeticError
            if counter > 5:
                cv2.putText(image, "Please take rest", 
                           (300,150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 5, (255,0,0), 8, cv2.LINE_AA
                                )
                # Press 's' to reset counter
                if cv2.waitKey(10) & 0xFF == ord('s'):
                    counter = 0

            # Check if left or right shoulder angle is more than 45 degree notify "low your hand down"
            if angle_shoulder_left > 45 and angle_shoulder_right > 45:
                cv2.putText(image, "Do not go up too high with your hands.", 
                           (150,300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
                cv2.putText(image, "Tuck your shoulders in!", 
                           (150,400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
            elif angle_shoulder_left > 45:
                cv2.putText(image, "Do not go up too high with your left hand.", 
                           (150,300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
                cv2.putText(image, "Tuck your shoulders in!", 
                           (150,400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
            elif angle_shoulder_right > 45:
                cv2.putText(image, "Do not go up too high with your right hand.", 
                           (150,300), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
                cv2.putText(image, "Tuck your shoulders in!", 
                           (150,400), 
                           cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0,0,255), 6, cv2.LINE_AA
                                )   
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'CNT', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Fitful Model Application - Both Hands Exercise', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1705463230.915187       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro


: 